# 5.2.3 Representación TF-IDF

In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer

import warnings
warnings.filterwarnings("ignore")

En una matriz documento-término, las cantidades representan la ocurrencia de un token en cada uno de los documentos.

Las palabras como artículos, verbos ser/estar, conectores, etc. son palabras muy comunes en los textos (tienen frecuencias altas en una matriz documento-término) y tienen poca utilidad para extraer información valiosa de un documento. Adicionalmente, distorcionan y obscurecen términos interesantes que serían de mucha más utilidad.

La representación TF-IDF (term-frequency inverse document-frequency) recomputa los valores de la matriz como:

$$ tf-idf(t,d) = tf(t,d) x idf(t) $$

- $t$ representa el término.
- $d$ representa el documento.

La transformación también pondera la frecuencia de cada token respecto a su frecuencia en el documento y la cantidad de tokens en dicho documento.

Para explicar esta representación considere la siguiente matriz documento-termino:

In [8]:
#
# tf: term-frequency
#
tf = np.array(
    [
        [3, 0, 1],
        [2, 0, 0],
        [3, 0, 0],
        [4, 0, 0],
        [3, 2, 0],
        [3, 0, 2],
    ]
)

tf

array([[3, 0, 1],
       [2, 0, 0],
       [3, 0, 0],
       [4, 0, 0],
       [3, 2, 0],
       [3, 0, 2]])

Ahora se crea el transformador:

In [9]:
transformer = TfidfTransformer(
    # -------------------------------------------------------------
    # Each output row will have unit norm.
    # "l1", "l2"
    norm="l1",
    # -------------------------------------------------------------
    # Enable inverse-document-frequency reweighting.
    use_idf=True,
    # -------------------------------------------------------------
    # Smooth idf weights by adding one to document frequencies, as
    # if an extra document was seen containing every term in the
    # collection exactly once. Prevents zero divisions.
    smooth_idf=False,
    # -------------------------------------------------------------
    # Apply sublinear tf scaling, i.e. replace tf with 1 + log(tf).
    sublinear_tf=False,
)

transformer.fit_transform(tf).toarray()

array([[0.5883954 , 0.        , 0.4116046 ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.34950701, 0.65049299, 0.        ],
       [0.41682734, 0.        , 0.58317266]])

## 5.2.3.1 Caso 1

* norm="l1"

* use_idf=False

In [3]:
#
# term-frequency
#
tf

array([[3, 0, 1],
       [2, 0, 0],
       [3, 0, 0],
       [4, 0, 0],
       [3, 2, 0],
       [3, 0, 2]])

In [4]:
#
# norma "l1" de cada fila
#
row_norm = np.tile(tf.sum(axis=1).reshape(-1, 1), (1, 3))
row_norm

array([[4, 4, 4],
       [2, 2, 2],
       [3, 3, 3],
       [4, 4, 4],
       [5, 5, 5],
       [5, 5, 5]])

In [5]:
tf / row_norm

array([[0.75, 0.  , 0.25],
       [1.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [0.6 , 0.4 , 0.  ],
       [0.6 , 0.  , 0.4 ]])

In [10]:
#
# Verificación
#
TfidfTransformer(
    norm="l1",
    use_idf=False,
    smooth_idf=False,
    sublinear_tf=False,
).fit_transform(tf).toarray()

array([[0.75, 0.  , 0.25],
       [1.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [0.6 , 0.4 , 0.  ],
       [0.6 , 0.  , 0.4 ]])

## 5.2.3.2 Caso 2

* norm="l2"

* use_idf=False

In [11]:
#
# term-frequency
#
tf

array([[3, 0, 1],
       [2, 0, 0],
       [3, 0, 0],
       [4, 0, 0],
       [3, 2, 0],
       [3, 0, 2]])

In [12]:
#
# norma "l2" de cada fila
#
row_norm = np.tile(np.sqrt(np.power(tf, 2).sum(axis=1).reshape(-1, 1)), (1, 3))
row_norm

array([[3.16227766, 3.16227766, 3.16227766],
       [2.        , 2.        , 2.        ],
       [3.        , 3.        , 3.        ],
       [4.        , 4.        , 4.        ],
       [3.60555128, 3.60555128, 3.60555128],
       [3.60555128, 3.60555128, 3.60555128]])

In [13]:
tf / row_norm

array([[0.9486833 , 0.        , 0.31622777],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.83205029, 0.5547002 , 0.        ],
       [0.83205029, 0.        , 0.5547002 ]])

In [14]:
#
# Verificación
#
TfidfTransformer(
    norm="l2",
    use_idf=False,
    smooth_idf=False,
    sublinear_tf=False,
).fit_transform(tf).toarray()

array([[0.9486833 , 0.        , 0.31622777],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.83205029, 0.5547002 , 0.        ],
       [0.83205029, 0.        , 0.5547002 ]])

## 5.2.3.3 Caso 3

* norm="l2"

* use_idf=True

* smooth_idf=False

* sublinear_tf=False

In [15]:
#
# número de documentos = 6
#
n = tf.shape[0]

In [16]:
#
# Cuenta los documentos en que aparece un término
#
df = np.where(tf > 0, 1, 0)
df = df.sum(axis=0)
df

array([6, 1, 2])

In [17]:
#
# Computa idf(t). Para smooth_idf=False
#
#                  n
#   idf(t) = log ---- + 1
#                 df
#
idf = np.log(n / df) + 1
idf = np.tile(idf, (6, 1))
idf

array([[1.        , 2.79175947, 2.09861229],
       [1.        , 2.79175947, 2.09861229],
       [1.        , 2.79175947, 2.09861229],
       [1.        , 2.79175947, 2.09861229],
       [1.        , 2.79175947, 2.09861229],
       [1.        , 2.79175947, 2.09861229]])

In [18]:
tf_idf_raw = tf * idf
tf_idf_raw

array([[3.        , 0.        , 2.09861229],
       [2.        , 0.        , 0.        ],
       [3.        , 0.        , 0.        ],
       [4.        , 0.        , 0.        ],
       [3.        , 5.58351894, 0.        ],
       [3.        , 0.        , 4.19722458]])

In [19]:
row_norm = np.tile(np.sqrt(np.power(tf_idf_raw, 2).sum(axis=1).reshape(-1, 1)), (1, 3))
row_norm

array([[3.66117106, 3.66117106, 3.66117106],
       [2.        , 2.        , 2.        ],
       [3.        , 3.        , 3.        ],
       [4.        , 4.        , 4.        ],
       [6.33842912, 6.33842912, 6.33842912],
       [5.15913696, 5.15913696, 5.15913696]])

In [20]:
tf_idf = tf_idf_raw / row_norm
tf_idf

array([[0.81940995, 0.        , 0.57320793],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.47330339, 0.88089948, 0.        ],
       [0.58149261, 0.        , 0.81355169]])

In [21]:
#
# Verificación
#
TfidfTransformer(
    norm="l2",
    use_idf=True,
    smooth_idf=False,
    sublinear_tf=False,
).fit_transform(tf).toarray()

array([[0.81940995, 0.        , 0.57320793],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.47330339, 0.88089948, 0.        ],
       [0.58149261, 0.        , 0.81355169]])

## 5.2.3.4 Caso 4

* norm="l2"

* use_idf=True

* smooth_idf=True

* sublinear_tf=False

In [22]:
#
# Computa idf(t). Para smooth_idf=True. Equivale
# a un documento que tiene todos los terminos
#
#                  1+n
#   idf(t) = log ------ + 1
#                 1+df
#
idf = np.log((1 + n) / (1 + df)) + 1
idf = np.tile(idf, (6, 1))
idf

array([[1.        , 2.25276297, 1.84729786],
       [1.        , 2.25276297, 1.84729786],
       [1.        , 2.25276297, 1.84729786],
       [1.        , 2.25276297, 1.84729786],
       [1.        , 2.25276297, 1.84729786],
       [1.        , 2.25276297, 1.84729786]])

In [23]:
tf_idf_raw = tf * idf
tf_idf_raw

array([[3.        , 0.        , 1.84729786],
       [2.        , 0.        , 0.        ],
       [3.        , 0.        , 0.        ],
       [4.        , 0.        , 0.        ],
       [3.        , 4.50552594, 0.        ],
       [3.        , 0.        , 3.69459572]])

In [24]:
row_norm = np.tile(np.sqrt(np.power(tf_idf_raw, 2).sum(axis=1).reshape(-1, 1)), (1, 3))
row_norm

array([[3.52313914, 3.52313914, 3.52313914],
       [2.        , 2.        , 2.        ],
       [3.        , 3.        , 3.        ],
       [4.        , 4.        , 4.        ],
       [5.41292564, 5.41292564, 5.41292564],
       [4.75920556, 4.75920556, 4.75920556]])

In [25]:
tf_idf = tf_idf_raw / row_norm
tf_idf

array([[0.85151335, 0.        , 0.52433293],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.55422893, 0.83236428, 0.        ],
       [0.63035731, 0.        , 0.77630514]])

In [26]:
#
# Verificación
#
TfidfTransformer(
    norm="l2",
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=False,
).fit_transform(tf).toarray()

array([[0.85151335, 0.        , 0.52433293],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.55422893, 0.83236428, 0.        ],
       [0.63035731, 0.        , 0.77630514]])

## 5.2.3.5 Caso 5

* norm="l2"

* use_idf=True

* smooth_idf=True

* sublinear_tf=True

In [27]:
#
# Computa idf(t). Para smooth_idf=True. Equivale
# a un documento que tiene todos los terminos
#
#                  1+n
#   idf(t) = log ------ + 1
#                 1+df
#
idf = np.log((1 + n) / (1 + df)) + 1
idf = np.tile(idf, (6, 1))
idf

array([[1.        , 2.25276297, 1.84729786],
       [1.        , 2.25276297, 1.84729786],
       [1.        , 2.25276297, 1.84729786],
       [1.        , 2.25276297, 1.84729786],
       [1.        , 2.25276297, 1.84729786],
       [1.        , 2.25276297, 1.84729786]])

In [28]:
#
# Cuando sublinear_tf=True, reemplaza tf por 1 + log(tf)
#
mylog = lambda x: 1 + np.log(x) if x > 0 else 0

tf_idf_raw = np.vectorize(mylog)(tf) * idf
tf_idf_raw

array([[2.09861229, 0.        , 1.84729786],
       [1.69314718, 0.        , 0.        ],
       [2.09861229, 0.        , 0.        ],
       [2.38629436, 0.        , 0.        ],
       [2.09861229, 3.81425927, 0.        ],
       [2.09861229, 0.        , 3.12774716]])

In [29]:
row_norm = np.tile(np.sqrt(np.power(tf_idf_raw, 2).sum(axis=1).reshape(-1, 1)), (1, 3))
row_norm

array([[2.79583314, 2.79583314, 2.79583314],
       [1.69314718, 1.69314718, 1.69314718],
       [2.09861229, 2.09861229, 2.09861229],
       [2.38629436, 2.38629436, 2.38629436],
       [4.35347531, 4.35347531, 4.35347531],
       [3.76656022, 3.76656022, 3.76656022]])

In [30]:
tf_idf = tf_idf_raw / row_norm
tf_idf

array([[0.75062144, 0.        , 0.66073252],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.48205448, 0.87614124, 0.        ],
       [0.55716945, 0.        , 0.83039882]])

In [31]:
#
# Verificación
#
TfidfTransformer(
    norm="l2",
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=True,
).fit_transform(tf).toarray()

array([[0.75062144, 0.        , 0.66073252],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.48205448, 0.87614124, 0.        ],
       [0.55716945, 0.        , 0.83039882]])

In [12]:
print('ok_')

ok_
